# KO Bot v0.1
### Overview: 
1. Ingest **live market data** on a token pair; 
2. Create a [Hummingbot Pure Market Making strategy config](https://docs.hummingbot.org/strategies/pure-market-making/#strategy-configs) that corresponds to the data.

_TODO:_
> A good market making strategy is nothing more than planning ahead on how you will react to market changes.

[Hummingbot](https://blog.hummingbot.org/2020-09-what-is-market-making/)

## Part 1: Ingest Live Market Data
* [ticker data](https://docs.ccxt.com/#/README?id=ticker-structure)
* [order book](https://docs.ccxt.com/#/README?id=order-book)

In [4]:
import ccxt
import json

kucoin = ccxt.kucoin()
# kucoin.set_sandbox_mode(True)
kucoin.load_markets()

BTC = "BTC/USDT"

live_ticker = kucoin.fetch_ticker(BTC)
print(json.dumps(live_ticker, indent=2))

{
  "symbol": "BTC/USDT",
  "timestamp": 1689983624129,
  "datetime": "2023-07-21T23:53:44.129Z",
  "high": 30059.6,
  "low": 29727.1,
  "bid": 29908.7,
  "bidVolume": null,
  "ask": 29908.8,
  "askVolume": null,
  "vwap": 29861.21727180732,
  "open": 29796.3,
  "close": 29908.7,
  "last": 29908.7,
  "previousClose": null,
  "change": 112.4,
  "percentage": 0.37,
  "average": null,
  "baseVolume": 1992.93290837,
  "quoteVolume": 59511402.58497144,
  "info": {
    "time": 1689983624129,
    "symbol": "BTC-USDT",
    "buy": "29908.7",
    "sell": "29908.8",
    "changeRate": "0.0037",
    "changePrice": "112.4",
    "high": "30059.6",
    "low": "29727.1",
    "vol": "1992.93290837",
    "volValue": "59511402.584971441",
    "last": "29908.7",
    "averagePrice": null,
    "takerFeeRate": "0.001",
    "makerFeeRate": "0.001",
    "takerCoefficient": "1",
    "makerCoefficient": "1"
  }
}


In [5]:
live_orderbook = kucoin.fetch_order_book(BTC)
print(json.dumps(live_orderbook, indent=1))

{
 "symbol": "BTC/USDT",
 "bids": [
  [
   29908.7,
   3.62792629
  ],
  [
   29908.6,
   0.00283734
  ],
  [
   29908.5,
   0.00283734
  ],
  [
   29908.3,
   0.04463411
  ],
  [
   29907.9,
   0.01134936
  ],
  [
   29907.5,
   1.2
  ],
  [
   29907.1,
   0.33198733
  ],
  [
   29907.0,
   0.01688313
  ],
  [
   29906.6,
   0.09916285
  ],
  [
   29905.9,
   0.01650996
  ],
  [
   29905.7,
   0.00168893
  ],
  [
   29905.5,
   0.04539744
  ],
  [
   29905.3,
   0.50157379
  ],
  [
   29904.9,
   0.06596765
  ],
  [
   29904.3,
   0.12297146
  ],
  [
   29903.8,
   0.34682081
  ],
  [
   29902.5,
   0.03780903
  ],
  [
   29902.4,
   1.00326
  ],
  [
   29902.3,
   0.10751264
  ],
  [
   29901.9,
   0.50163083
  ],
  [
   29900.9,
   0.00031111
  ],
  [
   29900.2,
   0.32096159
  ],
  [
   29900.1,
   0.32663928
  ],
  [
   29898.9,
   7.606e-05
  ],
  [
   29898.6,
   0.01671748
  ],
  [
   29897.2,
   0.11080955
  ],
  [
   29896.7,
   7.606e-05
  ],
  [
   29895.9,
   0.33193764
 

In [6]:
best_bid = live_orderbook["bids"][0][0] if len(live_orderbook["bids"]) > 0 else None
best_ask = live_orderbook["asks"][0][0] if len(live_orderbook["asks"]) > 0 else None
spread = (best_ask - best_bid) if (best_bid and best_ask) else None
print(
    kucoin.id,
    "market price",
    {"bid": best_bid, "ask": best_ask, "spread": spread}
)

kucoin market price {'bid': 29908.7, 'ask': 29908.8, 'spread': 0.09999999999854481}


## Part 2: Generate Hummingbot PMM Config

**2.a: Determine the minimum requirments for a fully-fledged PMM Script (PMM MVP)**

[This file](./pmm_config_vars.yml) contains all the HB configs that can be included, retrofitted with `required`, `promptable`, and `default` (if applicable)